In [ ]:
!nvidia-smi

Thu Feb 29 05:33:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image

# Testes

In [ ]:
def load_mobilenetv2():
    #local_weights_file = MODELS_PATH + 'mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_128_no_top.h5'
    pre_trained_model = MobileNetV2(input_shape = (120, 160, 3),
                                    include_top = False)
                                    #,weights = None)

    #pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    '''
      POR QUÊ?????????
    '''
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('out_relu')
    print('last layer output shape: ', last_layer.output_shape)
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics='accuracy')

    return model

In [ ]:
class accCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
modelo = load_mobilenetv2()

9406464/9406464 [==============================] - 0s 0us/step
last layer output shape:  (None, 4, 5, 1280)


In [ ]:
history_mobilenet_cpu = modelo.fit(
    train_dataset(),
    epochs=5,
    batch_size=32,
    validation_data=test_dataset(),
    callbacks=[accCallback()]
)

Found 2926 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.
Epoch 1/5
 6/92 [>.............................] - ETA: 8:33 - loss: 9.8660 - accuracy: 0.6354 

In [ ]:
modelo.summary()

In [ ]:
hidden_layers = modelo.layers[1:-1]  # Exclude input and output layers

# Print information about each hidden layer
for layer in hidden_layers:
    print("Layer Name:", layer.name)
    print("Layer Type:", type(layer))
    print("Layer Configuration:", layer.get_config())
    print("Layer Weights:", layer.get_weights())
    print()

In [ ]:
model_mobilenet_cpu = load_mobilenetv2()

9406464/9406464 [==============================] - 1s 0us/step
last layer output shape:  (None, 4, 5, 1280)


In [ ]:
history_mobilenet_cpu = model_mobilenet_cpu.fit(
    train_dataset(),
    epochs=5,
    batch_size=32,
    validation_data=test_dataset(),
    callbacks=[accCallback()]
)

Found 2926 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.
Epoch 1/5
92/92 [==============================] - 893s 10s/step - loss: 1.2146 - accuracy: 0.8435 - val_loss: 0.0656 - val_accuracy: 0.9752
Epoch 2/5
92/92 [==============================] - 182s 2s/step - loss: 0.1393 - accuracy: 0.9607 - val_loss: 0.0229 - val_accuracy: 0.9920
Epoch 3/5
92/92 [==============================] - 174s 2s/step - loss: 0.0509 - accuracy: 0.9863 - val_loss: 0.7802 - val_accuracy: 0.7780
Epoch 4/5
92/92 [==============================] - 231s 3s/step - loss: 0.0184 - accuracy: 0.9938 - val_loss: 0.0094 - val_accuracy: 0.9968
Epoch 5/5
92/92 [==============================] - 179s 2s/step - loss: 0.0585 - accuracy: 0.9884 - val_loss: 0.0183 - val_accuracy: 0.9944


In [ ]:
model_mobilenet_gpu = load_mobilenetv2()

last layer output shape:  (None, 4, 5, 1280)


In [ ]:
history_mobilenet_gpu = model_mobilenet_gpu.fit(
    train_dataset(),
    epochs=5,
    batch_size=32,
    validation_data=test_dataset(),
    callbacks=[accCallback()]
)

Found 2926 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.
Epoch 1/5
92/92 [==============================] - 313s 3s/step - loss: 1.3152 - accuracy: 0.8712 - val_loss: 0.1747 - val_accuracy: 0.9441
Epoch 2/5
92/92 [==============================] - 63s 690ms/step - loss: 0.1883 - accuracy: 0.9484 - val_loss: 0.1624 - val_accuracy: 0.9497
Epoch 3/5
92/92 [==============================] - 66s 717ms/step - loss: 0.0516 - accuracy: 0.9856 - val_loss: 0.0983 - val_accuracy: 0.9704
Epoch 4/5
92/92 [==============================] - 65s 710ms/step - loss: 0.0586 - accuracy: 0.9911 - val_loss: 0.0166 - val_accuracy: 0.9952
Epoch 5/5
92/92 [==============================] - 65s 707ms/step - loss: 0.0467 - accuracy: 0.9932 - val_loss: 0.0238 - val_accuracy: 0.9928


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5963656820392819886
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17178283617951481328
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [ ]:
tf.debugging.set_log_device_placement(False)

# VGG-16

In [ ]:
def load_vgg16():
    #local_weights_file = MODELS_PATH + 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    pre_trained_model = VGG16(input_shape = (120, 160, 3),
                                    include_top = False)
                                    #,weights = None)

    #pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    '''
      POR QUÊ?????????
    '''
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('block5_pool')
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics='accuracy')

    return model

In [ ]:
model_vgg = load_vgg16()

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
history_vgg = model_vgg.fit(
    train_dataset(),
    epochs=5,
    batch_size=32,
    validation_data=test_dataset(),
    callbacks=[accCallback()]
)

In [ ]:
model_vgg2 = load_vgg16()

In [ ]:
history_vgg2 = model_vgg2.fit(
    train_dataset(),
    epochs=5,
    batch_size=32,
    validation_data=test_dataset(),
    callbacks=[accCallback()]
)

Found 2926 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.
Epoch 1/5
92/92 [==============================] - 69s 743ms/step - loss: 0.3839 - accuracy: 0.8465 - val_loss: 0.1310 - val_accuracy: 0.9561
Epoch 2/5
92/92 [==============================] - 65s 711ms/step - loss: 0.1193 - accuracy: 0.9586 - val_loss: 0.0563 - val_accuracy: 0.9896
Epoch 3/5
92/92 [==============================] - 66s 717ms/step - loss: 0.0629 - accuracy: 0.9805 - val_loss: 0.1033 - val_accuracy: 0.9593
Epoch 4/5
92/92 [==============================] - 67s 728ms/step - loss: 0.0376 - accuracy: 0.9894 - val_loss: 0.0308 - val_accuracy: 0.9920
Epoch 5/5
92/92 [==============================] - 66s 717ms/step - loss: 0.0302 - accuracy: 0.9894 - val_loss: 0.0161 - val_accuracy: 0.9968


# MobileNetv2 2

In [ ]:
mobilenet_test2 = load_mobilenetv2()

9406464/9406464 [==============================] - 0s 0us/step
last layer output shape:  (None, 4, 5, 1280)


In [ ]:
history_mobilenet_test2 = mobilenet_test2.fit(
    train_dataset(),
    epochs=5,
    batch_size=32,
    validation_data=test_dataset(),
    callbacks=[accCallback()]
)

Found 2926 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.
Epoch 1/5
92/92 [==============================] - 72s 736ms/step - loss: 1.1167 - accuracy: 0.8616 - val_loss: 0.0550 - val_accuracy: 0.9768
Epoch 2/5
92/92 [==============================] - 68s 744ms/step - loss: 0.1113 - accuracy: 0.9692 - val_loss: 0.7400 - val_accuracy: 0.8099
Epoch 3/5
92/92 [==============================] - 65s 708ms/step - loss: 0.0768 - accuracy: 0.9781 - val_loss: 0.0370 - val_accuracy: 0.9944
Epoch 4/5
92/92 [==============================] - 66s 718ms/step - loss: 0.0722 - accuracy: 0.9809 - val_loss: 0.0200 - val_accuracy: 0.9944
Epoch 5/5
92/92 [==============================] - ETA: 0s - loss: 0.0202 - accuracy: 0.9956
Reached 99.5% accuracy so cancelling training!
92/92 [==============================] - 66s 721ms/step - loss: 0.0202 - accuracy: 0.9956 - val_loss: 0.0107 - val_accuracy: 0.9960


In [ ]:
mobilenet_test33333 = load_mobilenetv2()

last layer output shape:  (None, 4, 5, 1280)


In [ ]:
history_mobilenet_test33333 = mobilenet_test33333.fit(
    train_dataset(),
    epochs=5,
    batch_size=32,
    validation_data=test_dataset(),
    callbacks=[accCallback()]
)

Found 2926 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.
Epoch 1/5
92/92 [==============================] - 69s 725ms/step - loss: 1.2606 - accuracy: 0.8626 - val_loss: 0.0405 - val_accuracy: 0.9816
Epoch 2/5
92/92 [==============================] - 65s 714ms/step - loss: 0.1252 - accuracy: 0.9617 - val_loss: 0.0540 - val_accuracy: 0.9800
Epoch 3/5
92/92 [==============================] - 66s 722ms/step - loss: 0.1023 - accuracy: 0.9805 - val_loss: 0.0865 - val_accuracy: 0.9744
Epoch 4/5
92/92 [==============================] - 66s 724ms/step - loss: 0.0587 - accuracy: 0.9856 - val_loss: 0.0220 - val_accuracy: 0.9952
Epoch 5/5
92/92 [==============================] - 65s 707ms/step - loss: 0.0325 - accuracy: 0.9938 - val_loss: 0.0195 - val_accuracy: 0.9936


# Algoritmo Evolucionário

In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
import random
import numpy as np
import math
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

from deap import benchmarks

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=65)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
DATASET_TEST_PATH = '/content/drive/MyDrive/AIRTLab/test/'

DATASET_TRAIN_PATH = '/content/drive/MyDrive/AIRTLab/train/'

TARGET_SIZE = (120,160)

DESIRED_ACCURACY = 0.995

UPPER_BOUND = 155

LOWER_BOUND = 90

In [ ]:
def test_dataset():
    train_datagen = ImageDataGenerator(rescale = 1.0/255.)

    return train_datagen.flow_from_directory(DATASET_TEST_PATH,
                                             batch_size=32,
                                             class_mode='categorical',
                                             target_size=TARGET_SIZE)

def train_dataset():
    train_datagen = ImageDataGenerator(rescale = 1.0/255.)

    return train_datagen.flow_from_directory(DATASET_TRAIN_PATH,
                                             batch_size = 32,
                                             class_mode = 'categorical',
                                             target_size =TARGET_SIZE)

In [ ]:
train_set = train_dataset()
val_set = test_dataset()

Found 2926 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.


In [ ]:
lista_resultados_treinos = []

In [ ]:
def evaluate_individual(trainable_list):

  pre_trained_model = MobileNetV2(input_shape = (120, 160, 3), include_top = False)

  last_layer = pre_trained_model.get_layer('out_relu')
  last_output = last_layer.output

  #layers_to_set = pre_trained_model.layers

  for layer in pre_trained_model.layers:
        layer.trainable = False

  for layer, trainable in zip(pre_trained_model.layers[LOWER_BOUND:UPPER_BOUND], trainable_list):
    layer.trainable = bool(trainable)
    #print(layer.trainable)

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)
  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(1024, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x)
  # Add a final sigmoid layer for classification
  x = layers.Dense(2, activation='softmax')(x)

  model = Model(pre_trained_model.input, x)
  model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics='accuracy')

  history = model.fit(
    train_set,
    epochs=1,
    batch_size=32,
    validation_data=val_set,
    callbacks=[accCallback()]
  )

  #print(history.history['accuracy'])
  #print(len(history.history['accuracy']))

  lista_resultados_treinos.extend([pd.Series({key: history.history[key][i] for key in history.history}) for i in range(len(history.history['loss']))])

  return (history.history['val_accuracy'][0],) #return validation accuracy last result

In [ ]:
class accCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
import gc

class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [ ]:
tf.get_logger().setLevel('ERROR')

In [ ]:
tf.get_logger()

<Logger tensorflow (ERROR)>

In [ ]:
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxUniformPartialyMatched, indpb=0.1) #cxPartialyMatched  cxUniformPartialyMatched, indpb=0.1
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament,tournsize=4) #selRoulette  selTournament,tournsize=5

In [ ]:
def main():
  random.seed(64)

  pop = toolbox.population(n=10)
  hof = tools.HallOfFame(1)
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", np.mean)
  stats.register("std", np.std)
  stats.register("min", np.min)
  stats.register("max", np.max)


  pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=5,
                                   stats=stats, halloffame=hof, verbose=True)

  return pop, log, hof

In [ ]:
if __name__ == "__main__":
  results = main()

  gen = results[1].select('gen')
  best_per_gen = results[1].select('max')

  #print(results[2])

92/92 [==============================] - 99s 1s/step - loss: 0.4581 - accuracy: 0.9371 - val_loss: 1.8175 - val_accuracy: 0.6741
gen	nevals	avg     	std     	min     	max    
0  	10    	0.703275	0.118655	0.508786	0.88099
92/92 [==============================] - 100s 1s/step - loss: 0.3464 - accuracy: 0.9440 - val_loss: 1.0602 - val_accuracy: 0.7037
1  	8     	0.763978	0.110107	0.525559	0.890575
92/92 [==============================] - 97s 1s/step - loss: 0.3738 - accuracy: 0.9316 - val_loss: 0.5526 - val_accuracy: 0.8411
2  	8     	0.853195	0.111594	0.535942	0.953674
92/92 [==============================] - 96s 1s/step - loss: 0.4489 - accuracy: 0.9470 - val_loss: 1.6402 - val_accuracy: 0.6925
3  	4     	0.805192	0.147826	0.567891	0.953674
92/92 [==============================] - 100s 1s/step - loss: 0.3791 - accuracy: 0.9378 - val_loss: 0.6081 - val_accuracy: 0.8730
4  	4     	0.88139 	0.0637521	0.723642	0.953674
92/92 [==============================] - 98s 1s/step - loss: 0.4182 - ac

In [ ]:
generations = results[1]

In [ ]:
df = pd.DataFrame(generations)
df.to_csv('results_comp_evo.csv', index=False)

In [ ]:
pd.read_csv('results_comp_evo.csv')

,gen,nevals,avg,std,min,max
0,0,10,0.703275,0.118655,0.508786,0.880990
1,1,8,0.763978,0.110107,0.525559,0.890575
2,2,8,0.853195,0.111594,0.535942,0.953674
3,3,4,0.805192,0.147826,0.567891,0.953674
4,4,4,0.881390,0.063752,0.723642,0.953674
5,5,6,0.889617,0.051034,0.781949,0.953674


In [ ]:
lista_resultados_treinos

[loss            0.347536
 accuracy        0.938824
 val_loss        1.256574
 val_accuracy    0.773962
 dtype: float64,
 loss            0.381237
 accuracy        0.942926
 val_loss        2.218775
 val_accuracy    0.551917
 dtype: float64,
 loss            0.413093
 accuracy        0.925837
 val_loss        0.357232
 val_accuracy    0.879393
 dtype: float64,
 loss            0.368178
 accuracy        0.946343
 val_loss        3.627829
 val_accuracy    0.508786
 dtype: float64,
 loss            0.485963
 accuracy        0.922761
 val_loss        0.305822
 val_accuracy    0.880990
 dtype: float64,
 loss            0.336404
 accuracy        0.943267
 val_loss        1.623903
 val_accuracy    0.699681
 dtype: float64,
 loss            0.560618
 accuracy        0.938483
 val_loss        1.519155
 val_accuracy    0.605431
 dtype: float64,
 loss            0.432772
 accuracy        0.947368
 val_loss        1.213255
 val_accuracy    0.747604
 dtype: float64,
 loss            0.406686
 accur

In [ ]:
pd.DataFrame(lista_resultados_treinos).to_csv('lista_resultados.csv', index=False)